In [3]:
sn_kva = 1000
c_in_nF_km =420
vn_kv_SK = 12
max_i_ka_9 = .588 #0.054
max_i_ka_12b = .588 #0.09271
max_i_ka_12 = .588 #0.09271
max_i_ka_15b = .588 #0.09141
max_i_ka_15 = .588 #0.09141
q_kvarSK09=138*.4
q_kvarSK12 = 39*.4
q_kvarSK12b = 759*.4
q_kvarSK15b = 897*.4
q_kvarSK15 = 1112*.4
#http://electrical-engineering-portal.com/reactive-power-and-compensation-calculation-basics
# weighted avg http://nepsi.com/resources/calculators/calculation-of-cable-data.htm

In [6]:
import pandapower as pp #import panda power
import numpy as np
net=pp.create_empty_network("HA_LF",f_hz=50) #create an empty network #name, f_hz, sn_kva unsure of 50, 1000

## BUSES (temp vn_kv)
busSK = pp.create_bus(net, name="Bus SK", vn_kv=vn_kv_SK)
busSK09 = pp.create_bus(net, name="Bus SK09", vn_kv=11)
busSK12 = pp.create_bus(net, name="Bus SK12", vn_kv=11)
busSK12b = pp.create_bus(net, name="Bus SK12b", vn_kv=11) #PQ OR PU??? #VOLTAGE LIMITS CANT BE SET FOR SLACK BUSES?
busSK15b= pp.create_bus(net, name="Bus SK15b", vn_kv=10.5)
busSK15= pp.create_bus(net, name="Bus SK15", vn_kv=11)


## LINES change vals

b=3.6e-6 #b/2=c*10^-9*2pi*freq

#c_nf is line capacitance and Max_i is maximum thermal , max_i_ka=0.588, name="Line 1")
lineM0=pp.create_line_from_parameters(net, busSK, busSK09, length_km=.119, r_ohm_per_km=.13, x_ohm_per_km=.08, c_nf_per_km=c_in_nF_km, max_i_ka= max_i_ka_9, name="Line M0")
lineL0=pp.create_line_from_parameters(net, busSK, busSK12b, length_km=.1305, r_ohm_per_km=.151, x_ohm_per_km=.087, c_nf_per_km=388.5, max_i_ka= max_i_ka_12b, name="Line L0")
lineL1=pp.create_line_from_parameters(net, busSK12b, busSK12, length_km=.300, r_ohm_per_km=.12777, x_ohm_per_km=.08115, c_nf_per_km=c_in_nF_km, max_i_ka= max_i_ka_12, name="Line L1")
lineI0=pp.create_line_from_parameters(net, busSK, busSK15b, length_km=.480, r_ohm_per_km=.13, x_ohm_per_km=.08, c_nf_per_km=c_in_nF_km, max_i_ka= max_i_ka_15b, name="Line I0")
lineI1=pp.create_line_from_parameters(net, busSK15b, busSK15, length_km=.480, r_ohm_per_km=.13, x_ohm_per_km=.08, c_nf_per_km=c_in_nF_km, max_i_ka= max_i_ka_15, name="Line I1")

## LOADS (not sure which buses get the loads) q=reactivepower
load09=pp.create_load(net, busSK09, p_kw=138, q_kvar= q_kvarSK09,name="Load SK09") #voltage deendent load? or normal load? scaling factor? 
load12=pp.create_load(net, busSK12, p_kw=39, q_kvar=q_kvarSK12,name="Load SK12") 
load12b=pp.create_load(net, busSK12b, p_kw=759, q_kvar=q_kvarSK12b,name="Load SK12b") 
load15b=pp.create_load(net, busSK15b, p_kw=897, q_kvar=q_kvarSK15b,name="Load SK15b")
load15=pp.create_load(net, busSK15, p_kw=1112, q_kvar=q_kvarSK15,name="Load SK15")

## EV Loads
EVload09=pp.create_load(net, busSK09, p_kw=7, q_kvar= 0.4*7,name="EV Load SK09")
EVload12=pp.create_load(net, busSK12, p_kw=63, q_kvar= 0.4*63,name="EV Load SK12")
EVload12b=pp.create_load(net, busSK12b, p_kw=63, q_kvar= 0.4*63,name="EV Load SK12b")
EVload15b=pp.create_load(net, busSK15b, p_kw=966, q_kvar= 0.4*966,name="EV Load SK15b")
EVload15=pp.create_load(net, busSK15, p_kw=966, q_kvar= 0.4*966,name="EV Load SK15")


## EXTERNAL GRID Slack, change vals, assume top substation is slack
pp.create_ext_grid(net,busSK, voltage = 1.0, va_degree=0,name="SlackGrid")

## GENERATOR
#which type of generator applied here?
#gen1= pp.create_gen(net, bus2, p_kw=-219000, vm_pu=1.0, name="generator")
#gen2= pp.create_gen(net, bus6, p_kw=0, vm_pu=1.0, name="generator2")
# pp.create_gen(net, bus6, p_kw=-6000, max_q_kvar=3000, min_q_kvar=-3000, vm_pu=1.03, name="generator") 

## SHUNT no shunts?
#pp.create_shunt(net, bus8, q_kvar=-200, p_kw=0, name='Shunt')
net

This pandapower network includes the following parameter tables:
   - bus (6 elements)
   - load (10 elements)
   - ext_grid (1 element)
   - line (5 elements)

# Check the network



In [7]:
net.bus

,name,vn_kv,type,zone,in_service
0,Bus SK,12.0,b,None,True
1,Bus SK09,11.0,b,None,True
2,Bus SK12,11.0,b,None,True
3,Bus SK12b,11.0,b,None,True
4,Bus SK15b,10.5,b,None,True
5,Bus SK15,11.0,b,None,True


In [8]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,Line M0,None,0,1,0.1190,0.13000,0.08000,420.0,0.0,0.588,1.0,1,None,True
1,Line L0,None,0,3,0.1305,0.15100,0.08700,388.5,0.0,0.588,1.0,1,None,True
2,Line L1,None,3,2,0.3000,0.12777,0.08115,420.0,0.0,0.588,1.0,1,None,True
3,Line I0,None,0,4,0.4800,0.13000,0.08000,420.0,0.0,0.588,1.0,1,None,True
4,Line I1,None,4,5,0.4800,0.13000,0.08000,420.0,0.0,0.588,1.0,1,None,True


In [9]:
net.load

,name,bus,p_kw,q_kvar,const_z_percent,const_i_percent,sn_kva,scaling,in_service,type
0,Load SK09,1,138.0,55.2,0.0,0.0,NaN,1.0,True,None
1,Load SK12,2,39.0,15.6,0.0,0.0,NaN,1.0,True,None
2,Load SK12b,3,759.0,303.6,0.0,0.0,NaN,1.0,True,None
3,Load SK15b,4,897.0,358.8,0.0,0.0,NaN,1.0,True,None
4,Load SK15,5,1112.0,444.8,0.0,0.0,NaN,1.0,True,None
5,EV Load SK09,1,7.0,2.8,0.0,0.0,NaN,1.0,True,None
6,EV Load SK12,2,63.0,25.2,0.0,0.0,NaN,1.0,True,None
7,EV Load SK12b,3,63.0,25.2,0.0,0.0,NaN,1.0,True,None
8,EV Load SK15b,4,966.0,386.4,0.0,0.0,NaN,1.0,True,None
9,EV Load SK15,5,966.0,386.4,0.0,0.0,NaN,1.0,True,None


In [10]:
net.ext_grid

,name,bus,vm_pu,va_degree,in_service
0,SlackGrid,0,1.0,0.0,True


In [11]:
net.gen

,name,bus,p_kw,vm_pu,sn_kva,min_q_kvar,max_q_kvar,scaling,in_service,type


In [12]:
net.shunt

,bus,name,q_kvar,p_kw,vn_kv,step,max_step,in_service


# Solve

In [17]:
pp.runpp(net)

In [18]:
net

This pandapower network includes the following parameter tables:
   - bus (6 elements)
   - load (10 elements)
   - ext_grid (1 element)
   - line (5 elements)
 and the following results tables:
   - res_ext_grid (1 element)
   - res_load (10 elements)
   - res_bus (6 elements)
   - res_line (5 elements)

# Results

## Ybus

In [19]:
net._ppc["internal"]["Ybus"].todense()

matrix([[13911.71946858-8345.84520497j, -6751.54181844+4154.7949652 j,
             0.           +0.        j, -5486.35790764+3161.01415871j,
         -1673.81974249+1030.04291845j,     0.           +0.        j],
        [-6751.54181844+4154.7949652 j,  6751.54181844-4154.79383468j,
             0.           +0.        j,     0.           +0.        j,
             0.           +0.        j,     0.           +0.        j],
        [    0.           +0.        j,     0.           +0.        j,
          2249.35773322-1428.62232185j, -2249.35773322+1428.62471669j,
             0.           +0.        j,     0.           +0.        j],
        [-5486.35790764+3161.01415871j,     0.           +0.        j,
         -2249.35773322+1428.62471669j,  7735.71564087-4589.63533377j,
             0.           +0.        j,     0.           +0.        j],
        [-1673.81974249+1030.04291845j,     0.           +0.        j,
             0.           +0.        j,     0.           +0.        j,
  

## Voltages and Phase Angles

In [20]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-5020.830636,-1985.274156
1,0.999981,-0.000199,145.000000,58.000000
2,0.999805,-0.001802,102.000000,40.800000
3,0.999845,-0.001323,822.000000,328.800000
4,0.997869,-0.021406,1863.000000,745.200000
5,0.996399,-0.036116,2078.000000,831.200000


In [21]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,145.002606,55.740606,-145.000000,-58.000000,0.002606,-2.259394,0.007474,0.008197,0.008197,1.394038
1,924.138739,362.598939,-924.003765,-364.814397,0.134975,-2.215457,0.047763,0.052149,0.052149,8.868849
2,102.003765,36.014397,-102.000000,-40.800000,0.003765,-4.785603,0.005679,0.005767,0.005767,0.980806
3,3951.689290,1566.934611,-3943.852264,-1571.212577,7.837026,-4.277966,0.204527,0.233930,0.233930,39.784057
4,2080.852264,826.012577,-2078.000000,-831.200000,2.852264,-5.187423,0.123365,0.117893,0.123365,20.980481


In [22]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu


In [23]:
net.res_shunt

,p_kw,q_kvar,vm_pu


In [24]:
net.res_load

,p_kw,q_kvar
0,138.0,55.2
1,39.0,15.6
2,759.0,303.6
3,897.0,358.8
4,1112.0,444.8
5,7.0,2.8
6,63.0,25.2
7,63.0,25.2
8,966.0,386.4
9,966.0,386.4


In [25]:
net.res_ext_grid

,p_kw,q_kvar
0,-5020.830636,-1985.274156
